<a href="https://colab.research.google.com/github/Jakelinecs/Tareas-Machine-Learning/blob/main/N30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import shutil
import os

# Specify the directory to remove
extraction_dir = '/content/train' # Make sure this is the correct directory you want to remove

# Check if the directory exists before attempting to remove it
if os.path.exists(extraction_dir):
    shutil.rmtree(extraction_dir)
    print(f"Directory '{extraction_dir}' and its contents have been removed.")
else:
    print(f"Directory '{extraction_dir}' does not exist.")

Directory '/content/train' and its contents have been removed.


In [24]:
# Specify the path to your zip file
zip_file_path = 'train.zip' # Replace with the actual path to your zip file

# Specify the directory where you want to extract the contents
extraction_dir = '/content/train' # You can change this to your desired directory

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_dir, exist_ok=True)

# Open the zip file in read mode
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extraction_dir)

print(f"Successfully extracted '{zip_file_path}' to '{extraction_dir}'")

Successfully extracted 'train.zip' to '/content/train'


In [30]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dropout, concatenate, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize

# Check TensorFlow version
print(f"TensorFlow Version: {tf.__version__}")

# --- 1. Data preparation settings and generator ---
DATA_DIR = './train/'  # Usar dataset del 20% para pruebas rápidas
IMG_SIZE_ORIG = 101
IMG_SIZE_TARGET = 128
IMG_CHANNELS = 1

class TGSDataGenerator(Sequence):
    """Keras Data Generator for TGS Salt Challenge"""
    def __init__(self, df, batch_size=16, shuffle=True, **kwargs):
        super().__init__(**kwargs)
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = self.df.iloc[indexes]['id'].tolist()
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS), dtype=np.float32)
        y = np.empty((self.batch_size, IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS), dtype=np.float32)

        for i, id_name in enumerate(list_IDs_temp):
            img_path = os.path.join(DATA_DIR, 'images', id_name + '.png')
            mask_path = os.path.join(DATA_DIR, 'masks', id_name + '.png')

            try:
                img = imread(img_path)
                mask = imread(mask_path)
            except FileNotFoundError:
                # Crear datos dummy si no se encuentra el archivo
                img = np.random.randint(0, 255, (IMG_SIZE_TARGET, IMG_SIZE_TARGET), dtype=np.uint8)
                mask = np.random.randint(0, 255, (IMG_SIZE_TARGET, IMG_SIZE_TARGET), dtype=np.uint8)

            img = img[:, :, 0] if img.ndim == 3 else img
            mask = mask[:, :, 0] if mask.ndim == 3 else mask

            img_resized = resize(img, (IMG_SIZE_TARGET, IMG_SIZE_TARGET), mode='constant', preserve_range=True)
            mask_resized = resize(mask, (IMG_SIZE_TARGET, IMG_SIZE_TARGET), mode='constant', preserve_range=True)

            # Normalize images and masks, add channel dimension
            X[i,] = (img_resized / 255.0).reshape(IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS)
            y[i,] = (mask_resized / 255.0).reshape(IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS)

        return X, y

def load_tgs_data(data_dir=DATA_DIR, batch_size=16, test_size=0.2, val_size=0.2):
    """Load dataset and create generators"""
    try:
        # Get filenames from the images folder in data_dir
        all_files = [f.split('.')[0] for f in os.listdir(os.path.join(data_dir, 'images')) if f.endswith('.png')]
        df_ids = pd.DataFrame({'id': all_files})
    except FileNotFoundError:
        print("ERROR: TGS Salt dataset files not found in DATA_DIR. Please check the path.")
        return None, None, None

    if len(df_ids) == 0:
        print("ERROR: Image files not found.")
        return None, None, None

    # Data splitting
    train_df, temp_df = train_test_split(df_ids, test_size=(test_size + val_size), random_state=42)
    val_df, _ = train_test_split(temp_df, test_size=val_size/(test_size + val_size), random_state=42)

    print(f"Total images: {len(df_ids)}, Train: {len(train_df)}, Val: {len(val_df)}")

    train_gen = TGSDataGenerator(train_df, batch_size=batch_size, shuffle=True)
    val_gen = TGSDataGenerator(val_df, batch_size=batch_size, shuffle=False)

    return train_gen, val_gen

# --- 2. U-Net Decoder Blocks ---

def conv_block(input_tensor, num_filters):
    """Double Conv layers and BN layers for U-Net decoder"""
    x = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer='he_normal')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# --- 3. Encoder Definition (Transfer Learning) ---
def convert_to_3channel(input_tensor):
    return Conv2D(3, (1, 1), padding='same', name='input_3ch_conversion')(input_tensor)

def get_resnet_encoder(input_tensor, trainable_encoder=False):
    """Use ResNet50 as encoder"""
    input_3_channel = convert_to_3channel(input_tensor)

    base_model = ResNet50(weights='imagenet',
                          include_top=False,
                          input_shape=(IMG_SIZE_TARGET, IMG_SIZE_TARGET, 3))

    for layer in base_model.layers:
        layer.trainable = trainable_encoder

    x = base_model(input_3_channel)
    bottom_output = x
    skip_layers = []

    return bottom_output, skip_layers, bottom_output

def get_vgg_encoder(input_tensor, trainable_encoder=False):
    """Use VGG16 as encoder"""
    input_3_channel = convert_to_3channel(input_tensor)

    base_model = VGG16(weights='imagenet',
                       include_top=False,
                       input_shape=(IMG_SIZE_TARGET, IMG_SIZE_TARGET, 3))

    for layer in base_model.layers:
        layer.trainable = trainable_encoder

    x = base_model(input_3_channel)
    bottom_output = x
    skip_layers = []

    return bottom_output, skip_layers, bottom_output

# --- 4. U-Net Model Building Function ---

def build_transfer_unet(encoder_name='VGG16', trainable_encoder=False):
    """Build a U-Net model with the specified encoder"""
    input_shape = (IMG_SIZE_TARGET, IMG_SIZE_TARGET, IMG_CHANNELS)
    inputs = Input(shape=input_shape)

    # 1. Encoder selection and construction
    if encoder_name == 'VGG16':
        bottom_output, skip_layers, lowest_skip = get_vgg_encoder(inputs, trainable_encoder)
    elif encoder_name == 'ResNet50':
        bottom_output, skip_layers, lowest_skip = get_resnet_encoder(inputs, trainable_encoder)
    else:
        raise ValueError(f"Unknown encoder: {encoder_name}")

    # 2. Bottleneck layer
    x = conv_block(bottom_output, 512)

    # 3. Decoder construction - Both encoders should output 4x4 for IMG_SIZE_TARGET x IMG_SIZE_TARGET input
    # 4x4 -> 8x8
    x = UpSampling2D(size=(2, 2))(x)
    x = conv_block(x, 256)

    # 8x8 -> 16x16
    x = UpSampling2D(size=(2, 2))(x)
    x = conv_block(x, 128)

    # 16x16 -> 32x32
    x = UpSampling2D(size=(2, 2))(x)
    x = conv_block(x, 64)

    # 32x32 -> 64x64
    x = UpSampling2D(size=(2, 2))(x)
    x = conv_block(x, 32)

    # 64x64 -> 128x128
    x = UpSampling2D(size=(2, 2))(x)
    x = conv_block(x, 16)

    # 4. Final output layer
    x = Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(x)
    output = Conv2D(1, (1, 1), activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=output, name=f"U-Net_{encoder_name}")
    return model

# --- 5. IoU Metric and Main Execution Block ---

def mean_iou(y_true, y_pred):
    """Keras compatible Mean IoU metric"""
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)

    y_pred_f = tf.cast(y_pred_f > 0.5, tf.float32)

    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection

    epsilon = tf.keras.backend.epsilon()
    union = tf.maximum(union, epsilon)

    iou = intersection / union
    return iou

if __name__ == '__main__':
    BATCH_SIZE = 16
    EPOCHS = 3

    # 1. Load data generators
    train_gen, val_gen = load_tgs_data(batch_size=BATCH_SIZE)

    if train_gen is None:
        print("Skipping execution. Please check if the data is correctly placed.")
    else:
        # --- ResNet50 Model Training and Evaluation ---
        print("\n--- Training ResNet50 Encoder U-Net ---")
        resnet_model = build_transfer_unet(encoder_name='ResNet50', trainable_encoder=False)
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
        resnet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[mean_iou])

        print(f"ResNet50: Running training for {EPOCHS} epochs...")
        resnet_history = resnet_model.fit(
            train_gen,
            validation_data=val_gen,
            steps_per_epoch=len(train_gen),
            epochs=EPOCHS,
            verbose=1
        )
        resnet_iou = resnet_history.history['val_mean_iou'][-1]
        print(f"ResNet50 IoU (Final Val): {resnet_iou:.4f}")

        # --- VGG16 Model Training and Evaluation ---
        print("\n--- Training VGG16 Encoder U-Net ---")
        vgg_model = build_transfer_unet(encoder_name='VGG16', trainable_encoder=False)
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
        vgg_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[mean_iou])

        print(f"VGG16: Running training for {EPOCHS} epochs...")
        vgg_history = vgg_model.fit(
            train_gen,
            validation_data=val_gen,
            steps_per_epoch=len(train_gen),
            epochs=EPOCHS,
            verbose=1
        )
        vgg_iou = vgg_history.history['val_mean_iou'][-1]
        print(f"VGG16 IoU (Final Val): {vgg_iou:.4f}")

        # --- Result Comparison ---
        print("\n--- Comparison Results ---")
        print(f"ResNet50 Final Validation IoU: {resnet_iou:.4f}")
        print(f"VGG16 Final Validation IoU: {vgg_iou:.4f}")

        if resnet_iou > vgg_iou:
            print("=> ResNet50 showed higher accuracy with this initial setting.")
        elif vgg_iou > resnet_iou:
            print("=> VGG16 showed higher accuracy with this initial setting.")
        else:
            print("=> No significant difference in accuracy was observed.")


TensorFlow Version: 2.19.0
Total images: 800, Train: 480, Val: 160

--- Training ResNet50 Encoder U-Net ---
ResNet50: Running training for 3 epochs...
Epoch 1/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 236s 7s/step - loss: 12.1993 - mean_iou: 1.5916 - val_loss: 9.5455 - val_mean_iou: 0.9823
Epoch 2/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 254s 7s/step - loss: -39.5019 - mean_iou: 10.0373 - val_loss: -26.2061 - val_mean_iou: 8.9623
Epoch 3/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 257s 7s/step - loss: -76.3075 - mean_iou: 39.2950 - val_loss: -76.9791 - val_mean_iou: 41.5194
ResNet50 IoU (Final Val): 41.5194

--- Training VGG16 Encoder U-Net ---
VGG16: Running training for 3 epochs...
Epoch 1/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 354s 11s/step - loss: -13.3027 - mean_iou: 2.2141 - val_loss: -56.4645 - val_mean_iou: 4.3941
Epoch 2/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 380s 11s/step - loss: -156.7235 - mean_iou: 17.1759 - val_loss: -162.5457 - val_mean_iou: 7.4563
Epoch 3/3
30/30 ━━━━━━━━━━━━━━━━━━━━ 383s 11s/step - loss: -278.4838 - mean_iou: 3